### Что проверяем:

слова из списка сводеша сохраняют ранги своих соседей независимо от модели

слова из списка сводеша демонстрируют большую устойчивость к набору соседей, чем случайные слова

слова из списка сводеша сохраняют расстояния к частотным словам языка независимо от модели.


In [ ]:
# отсечение по рангу
5000 слов самых част
снизу - 
отфильтровать по частотности

посмотреть частоты списка Сводеша
женя глазунов - спросить про метрики

In [8]:
# coding: utf-8

'''
This module is formatting  user config with facts and entities,
creating entity index and producing all variants for entity-match
'''

import pymorphy2
import gensim

import re
import os
from tqdm import tqdm
from operator import itemgetter
from scipy import stats
import random
from functools import lru_cache
import pandas as pd
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
def open_model(m):
    if m.endswith('.vec'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
    elif m.endswith('.bin'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
    else:
        model = gensim.models.Word2Vec.load(m)
    return model

In [3]:
@lru_cache(maxsize=50)
def get_morph(token):
    return morph.parse(token)

In [4]:
def get_tagged(word):
    pos_dic = {"NOUN":["NOUN","PROPN"],    "ADJF":["ADJ"],    "ADJS":["ADJ"],    "COMP":["ADJ","ADV"],    "VERB":["VERB"],    "INFN":["VERB"],    "PRTF":["ADJ"],    "PRTS":["VERB"],    "GRND":["VERB"],    "NUMR":["NUM"],    "ADVB":["ADV"],    "NPRO":["PRON"],    "PRED":["ADV"],    "PREP":["ADP"],    "CONJ":["CCONJ","SCONJ"],    "PRCL":["PART"],    "INTJ":["INTJ"]}
    p = get_morph(word)[0]
    try:
        pos = pos_dic[p.tag.POS]
    except:
        return []
    lemma = p.normal_form
    return [lemma+'_'+tag for tag in pos]

In [5]:
def check_in_model(word, model):
    tag_list = get_tagged(word)
    for tag in tag_list:
        #тут реальное наличие
        if tag in model:
            return True
    return False

In [6]:
def get_svodesh_range(word, model, svodesh):
    syns = {}
    tag = get_tagged(word)
    
    if tag and tag[0] in model:
        syns = {s:model.similarity(tag[0],s) for s in svodesh}
    return sorted(syns.items(), key=itemgetter(1), reverse=True)

In [9]:
df = pd.read_csv('/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection.csv', sep='\t')
df.head()

,Unnamed: 0,Unnamed: 0.1,% intersection,svodesh/random,word
0,0,0.0,0.139013,svodesh,несколько_ADV
1,1,1.0,0.047244,svodesh,немного_ADV
2,2,2.0,0.026923,svodesh,мало_ADV
3,3,3.0,0.100437,svodesh,четыре_NUM
4,4,4.0,0.143519,svodesh,пять_NUM


In [10]:
#frequent
fr  = df[df['svodesh/random']=='freq']
#svodesh
sv = df[df['svodesh/random']=='svodesh']
#random
rn = df[df['svodesh/random']=='random']

# эксперимент 1 - получаем 10 ближайших соседей по каждой модели для каждого слова

по аранеа
по новостям
по нкря
по вики

затем для каждого слова считаем % пересечения соседей по моделям

In [13]:
dic_svodesh = {'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = { 'aranea':{},'taiga':{}, 'rnc':{}}
dic_freq = {'aranea':{},'taiga':{}, 'rnc':{}}

In [11]:
#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)

In [12]:
model = open_model(mpath)

In [14]:
num = 10

In [18]:
for w in list(fr['word']):
    if w not in dic_freq['aranea']:
        dic_freq['aranea'][w] = model.most_similar(w,topn=num)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
for w in list(sv['word']):
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=num)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [20]:
for w in list(rn['word']):
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=num)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [22]:
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['rnc']:
        try:
            dic_freq['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['rnc'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [23]:
for w in list(sv['word']):
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['rnc'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [27]:
for w in list(rn['word']):
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['rnc'][w] = []

In [28]:
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)

In [29]:
for w in list(fr['word']):
    if w not in dic_freq['taiga']:
        try:
            dic_freq['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['taiga'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [30]:
for w in list(sv['word']):
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['taiga'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [31]:
for w in list(rn['word']):
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['taiga'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [32]:
import pandas as pd


In [33]:
def count_set(list1, list2, list3):
    set1, set2, set3 = set(list1), set(list2), set(list3)
    d = set.intersection(set1, set2, set3)
    u = set(list(set1)+list(set2)+list(set3))
    return len(d)/len(u)

In [34]:
def get_words(dic):
    return [i[0] for i in dic]

In [35]:
rows_list = []
for word in list(sv['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in list(rn['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
for word in list(fr['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['taiga'][word]),get_words(dic_freq['aranea'][word]),get_words(dic_freq['rnc'][word]))
        rows_list.append(dict1)
df10 = pd.DataFrame(rows_list) 
df10.head()

,% intersection,svodesh/random,word
0,0.200000,svodesh,несколько_ADV
1,0.210526,svodesh,немного_ADV
2,0.041667,svodesh,мало_ADV
3,0.750000,svodesh,четыре_NUM
4,0.538462,svodesh,пять_NUM


In [36]:
df10['svodesh/random'].value_counts()

svodesh    173
random     173
freq       173
Name: svodesh/random, dtype: int64

In [52]:
#frequent
fr  = df10[df10['svodesh/random']=='freq']
#svodesh
sv = df10[df10['svodesh/random']=='svodesh']
#random
rn = df10[df10['svodesh/random']=='random']

In [38]:
#манн-уитни
from scipy.stats import mannwhitneyu

In [53]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=24112.5, pvalue=1.0)

In [54]:
#Получается, что нулевая гипотеза о том, что степень признака в выборках неравна (в первой меньше) неверна
#принимается альтернативная гипотеза, что в случайной выборке слов степень признака меньше

In [55]:
mannwhitneyu(sorted(sv['% intersection']), sorted(fr['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=14073.0, pvalue=0.1677503491735992)

In [56]:
mannwhitneyu(sorted(fr['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=23695.5, pvalue=1.0)

In [ ]:
# получается, что слова из списка сводеша более устойчивы к различным моделям, чем просто частотные слова
# а частотные слова более устойчивы, чем случайные

In [57]:
df10.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_10.csv', sep='\t')

## 20 ближайших

In [43]:
num = 20
dic_svodesh = {'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = { 'aranea':{},'taiga':{}, 'rnc':{}}
dic_freq = {'aranea':{},'taiga':{}, 'rnc':{}}

#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)
model = open_model(mpath)

for w in list(fr['word']):
    if w not in dic_freq['aranea']:
        dic_freq['aranea'][w] = model.most_similar(w,topn=num)
for w in list(sv['word']):
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=num)
for w in list(rn['word']):
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=num)
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['rnc']:
        try:
            dic_freq['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['rnc'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['rnc'][w] = []
for w in list(rn['word']):
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_random['rnc'][w] = []
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['taiga']:
        try:
            dic_freq['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['taiga'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['taiga'][w] = []
for w in list(rn['word']):
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['taiga'][w] = []
rows_list = []
for word in list(sv['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in list(rn['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
for word in list(fr['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['taiga'][word]),get_words(dic_freq['aranea'][word]),get_words(dic_freq['rnc'][word]))
        rows_list.append(dict1)
df20 = pd.DataFrame(rows_list) 
df20.head()

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,% intersection,svodesh/random,word
0,0.195122,svodesh,несколько_ADV
1,0.142857,svodesh,немного_ADV
2,0.061224,svodesh,мало_ADV
3,0.323529,svodesh,четыре_NUM
4,0.555556,svodesh,пять_NUM


In [46]:
#frequent
fr  = df20[df20['svodesh/random']=='freq']
#svodesh
sv = df20[df20['svodesh/random']=='svodesh']
#random
rn = df20[df20['svodesh/random']=='random']

In [47]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=20041.0, pvalue=0.9999999786626149)

In [48]:
mannwhitneyu(sorted(sv['% intersection']), sorted(fr['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=13684.0, pvalue=0.08434547546665477)

In [49]:
mannwhitneyu(sorted(fr['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=20912.5, pvalue=0.9999999999321715)

In [51]:
df20.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_20.csv', sep='\t')

## 50 ближайших

In [58]:
num = 50
dic_svodesh = {'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = { 'aranea':{},'taiga':{}, 'rnc':{}}
dic_freq = {'aranea':{},'taiga':{}, 'rnc':{}}

#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)
model = open_model(mpath)

for w in list(fr['word']):
    if w not in dic_freq['aranea']:
        dic_freq['aranea'][w] = model.most_similar(w,topn=num)
for w in list(sv['word']):
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=num)
for w in list(rn['word']):
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=num)
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['rnc']:
        try:
            dic_freq['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['rnc'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['rnc'][w] = []
for w in list(rn['word']):
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_random['rnc'][w] = []
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['taiga']:
        try:
            dic_freq['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['taiga'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['taiga'][w] = []
for w in list(rn['word']):
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['taiga'][w] = []
rows_list = []
for word in list(sv['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in list(rn['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
for word in list(fr['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['taiga'][word]),get_words(dic_freq['aranea'][word]),get_words(dic_freq['rnc'][word]))
        rows_list.append(dict1)
df50 = pd.DataFrame(rows_list) 
df50.head()

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,% intersection,svodesh/random,word
0,0.218750,svodesh,несколько_ADV
1,0.076923,svodesh,немного_ADV
2,0.039683,svodesh,мало_ADV
3,0.163462,svodesh,четыре_NUM
4,0.290698,svodesh,пять_NUM


In [59]:
#frequent
fr  = df50[df50['svodesh/random']=='freq']
#svodesh
sv = df50[df50['svodesh/random']=='svodesh']
#random
rn = df50[df50['svodesh/random']=='random']

In [60]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=21708.0, pvalue=0.9999999999997976)

In [61]:
mannwhitneyu(sorted(sv['% intersection']), sorted(fr['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=16260.0, pvalue=0.9182170025990256)

In [62]:
mannwhitneyu(sorted(fr['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=20636.0, pvalue=0.9999999994735176)

In [63]:
df50.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_50.csv', sep='\t')

## 100 ближайших

In [77]:
num = 100
dic_svodesh = {'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = { 'aranea':{},'taiga':{}, 'rnc':{}}
dic_freq = {'aranea':{},'taiga':{}, 'rnc':{}}

#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)
model = open_model(mpath)

for w in list(fr['word']):
    if w not in dic_freq['aranea']:
        dic_freq['aranea'][w] = model.most_similar(w,topn=num)
for w in list(sv['word']):
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=num)
for w in list(rn['word']):
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=num)
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['rnc']:
        try:
            dic_freq['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['rnc'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['rnc'][w] = []
for w in list(rn['word']):
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_random['rnc'][w] = []
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['taiga']:
        try:
            dic_freq['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['taiga'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['taiga'][w] = []
for w in list(rn['word']):
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['taiga'][w] = []
rows_list = []
for word in list(sv['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in list(rn['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
for word in list(fr['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['taiga'][word]),get_words(dic_freq['aranea'][word]),get_words(dic_freq['rnc'][word]))
        rows_list.append(dict1)
df100 = pd.DataFrame(rows_list) 
df100.head()

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,% intersection,svodesh/random,word
0,0.139013,svodesh,несколько_ADV
1,0.047244,svodesh,немного_ADV
2,0.026923,svodesh,мало_ADV
3,0.100437,svodesh,четыре_NUM
4,0.143519,svodesh,пять_NUM


In [78]:
#frequent
fr  = df100[df100['svodesh/random']=='freq']
#svodesh
sv = df100[df100['svodesh/random']=='svodesh']
#random
rn = df100[df100['svodesh/random']=='random']

In [80]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=20997.0, pvalue=0.9999999999560165)

In [81]:
mannwhitneyu(sorted(sv['% intersection']), sorted(fr['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=16564.5, pvalue=0.9573241345613146)

In [82]:
mannwhitneyu(sorted(fr['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=19967.5, pvalue=0.9999999626192643)

In [79]:
df100.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_100.csv', sep='\t')

## 200 ближайших

In [65]:
num = 200
dic_svodesh = {'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = { 'aranea':{},'taiga':{}, 'rnc':{}}
dic_freq = {'aranea':{},'taiga':{}, 'rnc':{}}

#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)
model = open_model(mpath)

for w in list(fr['word']):
    if w not in dic_freq['aranea']:
        dic_freq['aranea'][w] = model.most_similar(w,topn=num)
for w in list(sv['word']):
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=num)
for w in list(rn['word']):
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=num)
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['rnc']:
        try:
            dic_freq['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['rnc'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['rnc'][w] = []
for w in list(rn['word']):
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_random['rnc'][w] = []
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['taiga']:
        try:
            dic_freq['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['taiga'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['taiga'][w] = []
for w in list(rn['word']):
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['taiga'][w] = []
rows_list = []
for word in list(sv['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in list(rn['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
for word in list(fr['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['taiga'][word]),get_words(dic_freq['aranea'][word]),get_words(dic_freq['rnc'][word]))
        rows_list.append(dict1)
df200 = pd.DataFrame(rows_list) 
df200.head()

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,% intersection,svodesh/random,word
0,0.095137,svodesh,несколько_ADV
1,0.031481,svodesh,немного_ADV
2,0.024715,svodesh,мало_ADV
3,0.070981,svodesh,четыре_NUM
4,0.071429,svodesh,пять_NUM


In [66]:
#frequent
fr  = df200[df200['svodesh/random']=='freq']
#svodesh
sv = df200[df200['svodesh/random']=='svodesh']
#random
rn = df200[df200['svodesh/random']=='random']

In [67]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=19903.0, pvalue=0.9999999449646517)

In [68]:
mannwhitneyu(sorted(sv['% intersection']), sorted(fr['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=16765.5, pvalue=0.9735965927775658)

In [69]:
mannwhitneyu(sorted(fr['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=18608.0, pvalue=0.9999551951452422)

In [70]:
df200.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_200.csv', sep='\t')

## 300 ближайших

In [71]:
num = 300
dic_svodesh = {'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = { 'aranea':{},'taiga':{}, 'rnc':{}}
dic_freq = {'aranea':{},'taiga':{}, 'rnc':{}}

#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)
model = open_model(mpath)

for w in list(fr['word']):
    if w not in dic_freq['aranea']:
        dic_freq['aranea'][w] = model.most_similar(w,topn=num)
for w in list(sv['word']):
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=num)
for w in list(rn['word']):
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=num)
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['rnc']:
        try:
            dic_freq['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['rnc'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['rnc'][w] = []
for w in list(rn['word']):
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=num)
        except:
            dic_random['rnc'][w] = []
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)
for w in list(fr['word']):
    if w not in dic_freq['taiga']:
        try:
            dic_freq['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_freq['taiga'][w] = []
for w in list(sv['word']):
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=num)
        except:
            dic_svodesh['taiga'][w] = []
for w in list(rn['word']):
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=num)
        except:
            print(w)
            dic_random['taiga'][w] = []
rows_list = []
for word in list(sv['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in list(rn['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
for word in list(fr['word']):
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['taiga'][word]),get_words(dic_freq['aranea'][word]),get_words(dic_freq['rnc'][word]))
        rows_list.append(dict1)
df300 = pd.DataFrame(rows_list) 
df300.head()

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,% intersection,svodesh/random,word
0,0.072022,svodesh,несколько_ADV
1,0.025393,svodesh,немного_ADV
2,0.022642,svodesh,мало_ADV
3,0.060976,svodesh,четыре_NUM
4,0.047043,svodesh,пять_NUM


In [72]:
#frequent
fr  = df300[df300['svodesh/random']=='freq']
#svodesh
sv = df300[df300['svodesh/random']=='svodesh']
#random
rn = df300[df300['svodesh/random']=='random']

In [73]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=19149.0, pvalue=0.9999965807706422)

In [74]:
mannwhitneyu(sorted(sv['% intersection']), sorted(fr['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=16924.0, pvalue=0.98243566817002)

In [75]:
mannwhitneyu(sorted(fr['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=17673.0, pvalue=0.9982044460824449)

In [76]:
df300.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_300.csv', sep='\t')